## Homework 6: Intermarket prediction
### FINM 33150 (Winter 2025) - Quantitative Trading Strategies
#### NAME: Arturo Charleston
#### Student ID: 12449584

## Introduction
In this notebook,

In [ ]:
# PACKAGES
import os
import warnings
import nasdaqdatalink
import statsmodels.api as sm
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.interpolate import CubicSpline
from pyprojroot import here
from fredapi import Fred

# Global Options
warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:.5f}'.format

# Set directory
home_directory = "/Users/charleston/Downloads/git_repositories/finm-qts-2025"
os.chdir(home_directory)

# KEYS
from config import FRED_KEY
from config import QUANDL_KEY

nasdaqdatalink.ApiConfig.api_key = QUANDL_KEY

#from my_functions import *

# Dates
start_date = '2018-01-01'
end_date = "2024-12-31"

### Functions

### Data Preparing

In [50]:
# Read CDS data
cds = pd.read_csv(here('./data/Liq5YCDS.delim'), delimiter='\t', usecols=lambda column: column not in ['Unnamed: 0'])
cds['date'] = pd.to_datetime(cds['date'])
cds = cds.set_index('date')
tickers = cds['ticker'].unique()
tickers = np.append(tickers, 'SPY')

# Convert CDS data from long format to wide format, keeping only the 'parspread' column
cds = cds.pivot_table(index='date', columns='ticker', values='parspread')

cds_returns = cds.resample('W-WED').last().pct_change()

In [51]:
# Read the stock data for each ticker
tickers_data_list = []

for ticker in tickers:
    data = nasdaqdatalink.get_table(
        'QUOTEMEDIA/PRICES',
        date={'gte': start_date, 'lte': end_date},  # Specify start and end dates
        ticker=ticker,                             # Pass the ticker
        paginate=True
    )
    tickers_data_list.append(data[['date', 'ticker', 'close']])

# Concatenate all the dataframes into one
tickers_data = pd.concat(tickers_data_list).sort_values(by='date')

# Convert from long format to wide format
tickers_data = tickers_data.pivot(index='date', columns='ticker', values='close')

tickers_returns = tickers_data.resample('W-WED').last().pct_change()